In [ ]:
! pip install import-ipynb
! pip install ipynb
! pip install arch
! pip install numpy 
! pip install matplotlib
! pip install scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from arch.univariate import GARCH, ZeroMean
import pandas as pd 
from math import log 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import statsmodels.api as sm
import pickle


In [ ]:
with open('donnees_.pkl', 'rb') as f:
    donnees = pickle.load(f)

y = donnees['y']
r = donnees['r']
h = donnees['h']
echant_sim = donnees['echant_sim']
echant_reel = donnees['echant_reel']
echant_sim_10000 = donnees['echant_sim_10000']
echant_reel_10000 = donnees['echant_reel_10000']

## A. Approche naîve : Une seule régression

Avec des polynômes d'ordre 2, nous disposons de 9 variables de contrôle. Ces variables sont définies à partir des variables de contrôle d'ordre 1 et des paramètres omega.

L'approche naîve ZV-MCMC d'ordre 2 consiste en trois étapes :

1. Une courte simulation MCMC (de taille 2000) pour générer un échantillon omega, à partir duquel on génère un échantillon de variables de contrôle d'ordre 1 : CV1 ou Z (notation de l'article). Ces deux derniers sont ensuite utilisés pour générer un échantillon CV2 de 9 variables de contrôle d'ordre 2.

2. Une régression linéaire de omega sur les 9 variables CV2 est faite pour estimer les coefficients optimaux des variables CV2.

3. Une longue simulation MCMC (de taille 10000) pour générer un nouvel échantillon omega, CV1 ensuite CV2. On définit chaque $\tilde{\omega_i}$ comme la somme de $\omega_i$ et la somme des CV2 pondérée par les coefficients optimaux trouvés à l'étape 2.

$$\tilde{\omega_i} = \omega_i - \sum_{i=1}^{9} \hat{a}_i \times \text{CV2}_i$$ 

   où les $\hat{a}_i$ sont les coefficients de la régression linéaire de l'étape 2.

**Echantilllonnage des $\tilde{\omega_i}$** <br>
Pour les données simulées :

In [ ]:
def control_variates_2 (x,z):
    """
    Args:
        x (np.array): échantillon de paramètres (omega1, omega2, omega3)
        z (np.array): échantillon de variables de contrôle d'ordre 1 (z1,z2,z3)

    Returns:
        np.array: un échantillon de 9 variables de contrôle, ZV-MCMC d'ordre 2
    """
    i = np.ones(z.shape)
    u = z*x -0.5*i
    v1 = x[:,1]*z[:,0] + x[:,0]*z[:,1]
    v2 = x[:,2]*z[:,0] + x[:,0]*z[:,2]
    v3 = x[:,2]*z[:,1] + x[:,1]*z[:,2]
    v = np.column_stack((v1, v2, v3))
    vT = v.transpose()
    uT = u.transpose()
    zT = z.transpose()
    zTuTvT = np.concatenate((zT, uT, vT), axis = 0)

    # CV2 est un échantillon de même taille que x et z de 9-uplets ; Pour ZV-MCMC de degré 2 on a 9 variables de contrôles
    # Une colonne de CV2 est une variable de contôle 
    CV2 = zTuTvT.transpose()

    return CV2

In [ ]:
# 1. Courte et longue simulation des variables de contrôle d'ordre 2

CV2_sim = control_variates_2 (echant_sim,CV1_sim)
CV2_nv_sim = control_variates_2 (omega_nv_sim, CV1_nv_sim)

In [ ]:
# 2. Régression linéaire pour trouver les coefficients optimaux

def combi_omega(k, echant, CV_court, CV_long):
    """_summary_

    Args:
        k (int): 0, 1 ou 2 respectivement pour omega1, omega2 et omega3
        echant (np.array): un échantillon court des omega_i
        CV_court (np.array): un échantillon court de variables de contrôle à partir de 'échant'
        CV_long (np.array): un échantillon long de variables de contrôle indépendant de CV_court

    Returns:
        np.array: un échantillon long de combianisons linéaire des variables de contrôle pondérées par les coefficients optimaux
    """
    model = sm.OLS(echant[:,k], CV_court) 
    results = model.fit(cov_type='HC3') # On suppose l'hétéroscédasticité
    coeff_opt = results.params 
    combi = np.dot(coeff_opt, CV_long.transpose())
    print(results.summary())
    return combi

In [ ]:
# 3. Définition des omega_tilde après de longs échantillonnage MCMC

# ------------------------------------- Omega_1_tilde --------------------------------------------------------------

combi_omega1_d2 = combi_omega(0, echant_sim, CV2_sim, CV2_nv_sim)
omega_tilde_1_d2 = omega_nv_sim[:,0] - combi_omega1_d2
omega_tilde_1ss_d2 = enlever_outliers(combi_omega1_d2)

# ------------------------------------- Omega_1_tilde --------------------------------------------------------------

combi_omega2_d2 = combi_omega(1, echant_sim, CV2_sim, CV2_nv_sim)
omega_tilde_2_d2 = omega_nv_sim[:,1] - combi_omega2_d2
omega_tilde_2ss_d2 = enlever_outliers(omega_tilde_2_d2)

# ------------------------------------- Omega_1_tilde --------------------------------------------------------------

combi_omega3_d2 = combi_omega(2, echant_sim, CV2_sim, CV2_nv_sim)
omega_tilde_3_d2 = omega_nv_sim[:,2] - combi_omega3_d2
omega_tilde_3ss_d2 = enlever_outliers(omega_tilde_3_d2)

**Visualisation des trajectoires des $\tilde{\omega_i}$** <br>
Pour les données simulées :

In [ ]:

fig, axs = plt.subplots(3, 1, figsize=(8, 8))

axs[0].plot(range(9001), omega_nv_sim[:,0][1000:], label="Ordinary : {:.3f}".format(np.var(omega_nv_sim[1000:,0])), color = 'blue')
axs[0].plot(range(9001), omega_tilde_1ss[1000:], label="ZV 1 {:.3f}".format(np.var(omega_tilde_1ss[1000:])), color = 'red')
axs[0].plot(range(9001), omega_tilde_1ss_d2[1000:], label="ZV 2 {:.3f}".format(np.var(omega_tilde_1ss_d2[1000:])), color = 'grey')
axs[0].set_ylabel('w_tilde_1')
axs[0].legend()
axs[0].grid()

axs[1].plot(range(9001), omega_nv_sim[:,1][1000:], label="Ordinary : {:.3f}".format(np.var(omega_nv_sim[1000:,1])), color = 'blue')
axs[1].plot(range(9001), omega_tilde_2ss[1000:],  label="ZV 1 {:.3f}".format(np.var(omega_tilde_2ss[1000:])),  color = 'red')
axs[1].plot(range(9001), omega_tilde_2ss_d2[1000:], label="ZV 2 {:.2f}".format(np.var(omega_tilde_2ss_d2[1000:])), color = 'grey')
axs[1].set_ylabel('w_tilde_2')
axs[1].legend()
axs[1].grid()

axs[2].plot(range(9001), omega_nv_sim[:,2][1000:], label="Ordinary : {:.2f}".format(np.var(omega_nv_sim[1000:,2])), color = 'blue')
axs[2].plot(range(9001), omega_tilde_3ss[1000:],  label="ZV 1 {:.3f}".format(np.var(omega_tilde_3ss[1000:])), color = 'red')
axs[2].plot(range(9001), omega_tilde_3ss_d2[1000:], label="ZV 2 {:.4f}".format(np.var(omega_tilde_3ss_d2[1000:])), color = 'grey')
axs[2].set_ylabel('w_tilde_3')
axs[2].legend()
axs[2].grid()

plt.suptitle("Données simulées : \n Les trajectoires des omega_tildes avec MCMC Ordinary, ZV-MCMC d'ordre 1 et 2")
plt.show()

**Echantilllonnage des $\tilde{\omega_i}$** <br>
Pour les données réelles :

In [ ]:
CV2_reel = control_variates_2 (echant_reel,CV1_reel)
CV2_nv_reel = control_variates_2 (omega_nv_reel, CV1_nv_reel)

# ------------------------------------- Omega_1----------------------------------------------------------------

combi_omega1_d2_reel = combi_omega(0, echant_reel, CV2_reel, CV2_nv_reel)
omega_tilde_1_d2_reel = omega_nv_reel[:,0] - combi_omega1_d2_reel
omega_tilde_1ss_d2_reel = enlever_outliers(omega_tilde_1_d2_reel)

# ------------------------------------- Omega_2----------------------------------------------------------------

combi_omega2_d2_reel = combi_omega(1, echant_reel, CV2_reel, CV2_nv_reel)
omega_tilde_2_d2_reel = omega_nv_reel[:,1] - combi_omega2_d2_reel
omega_tilde_2ss_d2_reel = enlever_outliers(omega_tilde_2_d2_reel)

# ------------------------------------- Omega_3----------------------------------------------------------------

combi_omega3_d2_reel = combi_omega(2, echant_reel, CV2_reel, CV2_nv_reel)
omega_tilde_3_d2_reel = omega_nv_reel[:,2] - combi_omega3_d2_reel
omega_tilde_3ss_d2_reel = enlever_outliers(omega_tilde_3_d2_reel)

**Visualisation des trajectoires des $\tilde{\omega_i}$** <br>
Pour les données réelles :

In [ ]:

fig, axs = plt.subplots(3, 1, figsize=(8, 8))

axs[0].plot(range(9001), omega_nv_reel[1000:,0], label="Ordinary : {:.2f}".format(np.var(omega_nv_reel[1000:,0])), color = 'blue')
axs[0].plot(range(9001), omega_tilde_1ss_reel[1000:], label="ZV 1 {:.2f}".format(np.var(omega_tilde_1ss_reel[1000:])), color = 'red')
axs[0].plot(range(9001), omega_tilde_1ss_d2_reel[1000:], label="ZV 2 {:.4f}".format(np.var(omega_tilde_1ss_d2_reel[1000:])), color = 'grey')
axs[0].set_ylabel('w_tilde_1')
axs[0].legend()
axs[0].grid()

axs[1].plot(range(9001), omega_nv_reel[1000:,1], label="Ordinary : {:.2f}".format(np.var(omega_nv_reel[1000:,1])), color = 'blue')
axs[1].plot(range(9001), omega_tilde_2ss_reel[1000:], label="ZV 1 {:.3f}".format(np.var(omega_tilde_2ss_reel[1000:])),  color = 'red')
axs[1].plot(range(9001), omega_tilde_2ss_d2_reel[1000:], label="ZV 2 {:.4f}".format(np.var(omega_tilde_2ss_d2_reel[1000:])), color = 'grey')
axs[1].set_ylabel('w_tilde_2')
axs[1].legend()
axs[1].grid()

axs[2].plot(range(9001), omega_nv_reel[:,2][1000:], label="Ordinary : {:.3f}".format(np.var(omega_nv_reel[1000:,2])), color = 'blue')
axs[2].plot(range(9001), omega_tilde_3ss_reel[1000:], label="ZV 1 {:.3f}".format(np.var(omega_tilde_3ss_reel[1000:])), color = 'red')
axs[2].plot(range(9001), omega_tilde_3ss_d2_reel[1000:], label="ZV 2 {:.4f}".format(np.var(omega_tilde_3ss_d2_reel[1000:])), color = 'grey')
axs[2].set_ylabel('w_tilde_3')
axs[2].legend()
axs[2].grid()

plt.suptitle("Données réelles : \n Les trajectoires des omega_tildes avec MCMC Ordinary, ZV-MCMC d'ordre 1 et 2")
plt.show()

## B. Approche LASSO : double OLS

Cette approche en une régression pénalisée avec une partie des 9 variables de contrôle. Les variables de contrôle à garder sont déterminées avec les coefficients LASSO. Elle consiste en ces trois étapes.

1. Validation croisée pour sélectionner le meilleur paramètre de pénalité : celui qui minimise l'erreur quadratique moyenne

   On fait un grid search sur une plage prédéfinie de valeurs de $\lambda$ pour déterminer $\lambda^*$ qui minimise l'erreur quadratique moyenne MSE. Cette dernière est calculée pour chaque $\lambda$ avec une validation croisée sur l'échantillon court des 9 variables de contrôle. $$ \lambda^* = \arg\min_{\lambda} \text{MSE} (\lambda) $$

2. LASSO : Calcul de l'estimateur $\hat{\beta}_{\text{LASSO}}$ avec une régression pénalisée

   

   $$ \hat{\beta}_{\text{LASSO}} = \arg\min_{\beta} \left\{ \frac{1}{n} \sum_{i=1}^{n} (y_i - \beta^T x_i)^2 + \lambda^* ||\beta||_1 \right\} $$

3. Post LASSO : Régression linéaire non pénalisée sur les variables sélectionnées

   Les variables avec des coefficients de LASSO non nuls (supérieurs à $10^{-5}$ en valeur absolue) sont sélectionnées. Ensuite, un modèle de régression linéaire ordinaire est ajusté sur ces variables sélectionnées pour prédire les coefficients optimaux. Uniquement ces variables pondérées par les coefficients optimaux seront utilisées pour définir les $\tilde{\omega}_i$.

In [ ]:
# 1.a Calcul de l'erreur quadratique moyenne par validation croisée pour une valeur donnée de lambda

def cross_validation(X, Y, lamb):

    k = 10  # Nombre de plis
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    # Effectuer la validation croisée
    mse_scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        lasso = Lasso(alpha=lamb)
        lasso.fit(X_train_scaled, y_train)
        predictions = lasso.predict(X_test_scaled)

        # Calculer l'erreur quadratique moyenne (MSE) des prédictions
        mse = mean_squared_error(y_test, predictions)
        mse_scores.append(mse)

    # Calculer la moyenne des scores MSE sur les plis
    average_mse = np.mean(mse_scores)
    
    return average_mse

# 1.b Détermination de lambda_etoile par un grid search

def lambda_etoile (X,Y):
    MSE = []
    LAM  = np.linspace(0,100,1000)[1:]
    for lamb in LAM :
        MSE.append(cross_validation(X,Y,lamb))
    i = MSE.index(min(MSE))
    return LAM[i]

# 2. Calcul des coefficients du beta_LASSO

def beta_LASSO (X,Y, lamb) :
    lasso = Lasso(alpha=lamb)
    lasso.fit(X, Y)
    return lasso.coef_

# 3. Régression pénalisée qui estime les coefficients optimaux avec un échantillon court 
# et retourne un échantillon long des variables sélectionnées pondérées par les coefficcients optimaux.

def reg_non_penal(X_train, Y_train, X_nv, beta_LASSO):
    
    var = [] # Les variables sélectionnées par LASSO
    
    for i in range(len(beta_LASSO)) :
        if abs(beta_LASSO[i]) > 10**(-5) :
            var.append(i)

    model = LinearRegression()
    model.fit(X_train[:, var], Y_train)

    return model.predict(X_nv[:, var]) 


In [ ]:

lam_sim1 = lambda_etoile (CV2_sim, echant_sim[:,0])
beta_LASSO_sim1 = beta_LASSO (CV2_sim, echant_sim[:,0], lam_sim1)
omega_tilde_1_sim_LASSO = omega_nv_sim[:,0] - reg_penal(CV2_sim, echant_sim[:,0], CV2_nv_sim, beta_LASSO_sim1)
omega_tilde_1_sim_LASSO_ss = enlever_outliers(omega_tilde_1_sim_LASSO)

lam_sim2 = lambda_etoile (CV2_sim, echant_sim[:,1])
beta_LASSO_sim2 = beta_LASSO (CV2_sim, echant_sim[:,1], lam_sim2)
omega_tilde_2_sim_LASSO = omega_nv_sim[:,1] - reg_penal(CV2_sim, echant_sim[:,1], CV2_nv_sim, beta_LASSO_sim2)
omega_tilde_2_sim_LASSO_ss = enlever_outliers(omega_tilde_2_sim_LASSO)



lam_sim3 = lambda_etoile (CV2_sim, echant_sim[:,2])
beta_LASSO_sim3 = beta_LASSO (CV2_sim, echant_sim[:,2], lam_sim3)
omega_tilde_3_sim_LASSO = omega_nv_sim[:,2] - reg_penal(CV2_sim, echant_sim[:,2], CV2_nv_sim, beta_LASSO_sim3)
omega_tilde_3_sim_LASSO_ss = enlever_outliers(omega_tilde_3_sim_LASSO)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 8))

axs[0].plot(range(9001), omega_nv_sim[:,0][1000:], label="Ordinary : {:.3f}".format(np.var(omega_nv_sim[1000:,0])), color = 'blue')
axs[0].plot(range(9001), omega_tilde_1ss[1000:], label="ZV 1 {:.3f}".format(np.var(omega_tilde_1ss[1000:])), color = 'red')
axs[0].plot(range(9001), omega_tilde_1_sim_LASSO_ss[1000:], label="LASSO {:.4f}".format(np.var(omega_tilde_1_sim_LASSO_ss[1000:])), color = 'purple')

axs[0].set_ylabel('w_tilde_1')
axs[0].legend()

axs[1].plot(range(9001), omega_nv_sim[:,1][1000:], label="Ordinary : {:.3f}".format(np.var(omega_nv_sim[1000:,1])), color = 'blue')
axs[1].plot(range(9001), omega_tilde_2ss[1000:],  label="ZV 1 {:.3f}".format(np.var(omega_tilde_2ss[1000:])),  color = 'red')
axs[1].plot(range(9001), omega_tilde_2_sim_LASSO_ss[1000:], label="LASSO {:.3f}".format(np.var(omega_tilde_2_sim_LASSO_ss[1000:])), color = 'purple')

axs[1].set_ylabel('w_tilde_2')
axs[1].legend()

axs[2].plot(range(9001), omega_nv_sim[:,2][1000:], label="Ordinary : {:.2f}".format(np.var(omega_nv_sim[1000:,2])), color = 'blue')
axs[2].plot(range(9001), omega_tilde_3ss[1000:],  label="ZV 1 {:.3f}".format(np.var(omega_tilde_3ss[1000:])), color = 'red')
axs[2].plot(range(9001), omega_tilde_3_sim_LASSO_ss[1000:], label="LASSO {:.5f}".format(np.var(omega_tilde_3_sim_LASSO_ss[1000:])), color = 'purple')

axs[2].set_ylabel('w_tilde_3')
axs[2].legend()

plt.suptitle("Données simulées : \n Les trajectoires des omega_tildes avec MCMC ordianry, ZV d'ordre 1 et d'ordre 2 avec LASSO")
plt.grid()
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 8))

axs[0].plot(range(9001), omega_tilde_1ss_d2[1000:], label="ZV 2 {:.3f}".format(np.var(omega_tilde_1ss_d2[1000:])), color = 'grey')
axs[0].plot(range(9001), omega_tilde_1_sim_LASSO_ss[1000:], label="LASSO {:.4f}".format(np.var(omega_tilde_1_sim_LASSO_ss[1000:])), color = 'purple')
axs[0].set_ylabel('w_tilde_1')
axs[0].legend()

axs[1].plot(range(9001), omega_tilde_2ss_d2[1000:], label="ZV 2 {:.2f}".format(np.var(omega_tilde_2ss_d2[1000:])), color = 'grey')
axs[1].plot(range(9001), omega_tilde_2_sim_LASSO_ss[1000:], label="LASSO {:.3f}".format(np.var(omega_tilde_2_sim_LASSO_ss[1000:])), color = 'purple')
axs[1].set_ylabel('w_tilde_2')
axs[1].legend()

axs[2].plot(range(9001), omega_tilde_3ss_d2[1000:], label="ZV 2 {:.4f}".format(np.var(omega_tilde_3ss_d2[1000:])), color = 'grey')
axs[2].plot(range(9001), omega_tilde_3_sim_LASSO_ss[1000:], label="LASSO {:.5f}".format(np.var(omega_tilde_3_sim_LASSO_ss[1000:])), color = 'purple')
axs[2].set_ylabel('w_tilde_3')
axs[2].legend()

plt.suptitle("Données simulées : \n Les trajectoires des omega_tildes avec une régression non pénalisée et une régression pénalisée")
plt.grid()
plt.show()

In [ ]:
lam_reel1 = lambda_etoile (CV2_reel, echant_reel[:,0])
beta_LASSO_reel1 = beta_LASSO (CV2_reel, echant_reel[:,0], lam_reel1)
omega_tilde_1_reel_LASSO = omega_nv_reel[:,0] - reg_penal(CV2_reel, echant_reel[:,0], CV2_nv_reel, beta_LASSO_reel1)
omega_tilde_1_reel_LASSO_ss = enlever_outliers(omega_tilde_1_reel_LASSO)

lam_reel2 = lambda_etoile (CV2_reel, echant_reel[:,1])
beta_LASSO_reel2 = beta_LASSO (CV2_reel, echant_reel[:,1], lam_reel2)
omega_tilde_2_reel_LASSO = omega_nv_reel[:,1] - reg_penal(CV2_reel, echant_reel[:,1], CV2_nv_reel, beta_LASSO_reel2)
omega_tilde_2_reel_LASSO_ss = enlever_outliers(omega_tilde_2_reel_LASSO)



lam_reel3 = lambda_etoile (CV2_reel, echant_reel[:,2])
beta_LASSO_reel3 = beta_LASSO (CV2_reel, echant_reel[:,2], lam_reel3)
omega_tilde_3_reel_LASSO = omega_nv_reel[:,2] - reg_penal(CV2_reel, echant_reel[:,2], CV2_nv_reel, beta_LASSO_reel3)
omega_tilde_3_reel_LASSO_ss = enlever_outliers(omega_tilde_3_reel_LASSO)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 8))

axs[0].plot(range(9001), omega_nv_reel[:,0][1000:], label="Ordinary : {:.3f}".format(np.var(omega_nv_reel[1000:,0])), color = 'blue')
axs[0].plot(range(9001), omega_tilde_1ss_reel[1000:], label="ZV 1 {:.3f}".format(np.var(omega_tilde_1ss_reel[1000:])),  color = 'red')
axs[0].plot(range(9001), omega_tilde_1_reel_LASSO_ss[1000:], label="LASSO {:.4f}".format(np.var(omega_tilde_1_reel_LASSO_ss[1000:])), color = 'purple')

axs[0].set_ylabel('w_tilde_1')
axs[0].legend()

axs[1].plot(range(9001), omega_nv_sim[:,1][1000:], label="Ordinary : {:.3f}".format(np.var(omega_nv_sim[1000:,1])), color = 'blue')
axs[1].plot(range(9001), omega_tilde_2ss_reel[1000:], label="ZV 1 {:.3f}".format(np.var(omega_tilde_2ss_reel[1000:])),  color = 'red')
axs[1].plot(range(9001), omega_tilde_2_reel_LASSO_ss[1000:], label="LASSO {:.3f}".format(np.var(omega_tilde_2_reel_LASSO_ss[1000:])), color = 'purple')

axs[1].set_ylabel('w_tilde_2')
axs[1].legend()

axs[2].plot(range(9001), omega_nv_sim[:,2][1000:], label="Ordinary : {:.2f}".format(np.var(omega_nv_sim[1000:,2])), color = 'blue')
axs[2].plot(range(9001), omega_tilde_3ss_reel[1000:], label="ZV 1 {:.3f}".format(np.var(omega_tilde_3ss_reel[1000:])),  color = 'red')
axs[2].plot(range(9001), omega_tilde_3_reel_LASSO_ss[1000:], label="LASSO {:.5f}".format(np.var(omega_tilde_3_reel_LASSO_ss[1000:])), color = 'purple')

axs[2].set_ylabel('w_tilde_3')
axs[2].legend()

plt.suptitle("Données réelles : \n Les trajectoires des omega_tildes avec MCMC ordianry, ZV d'ordre 1 et d'ordre 2 avec LASSO")
plt.grid()
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 8))

axs[0].plot(range(9001), omega_tilde_1ss_d2_reel[1000:], label="ZV 2 {:.3f}".format(np.var(omega_tilde_1ss_d2_reel[1000:])), color = 'grey')
axs[0].plot(range(9001), omega_tilde_1_reel_LASSO_ss[1000:], label="LASSO {:.4f}".format(np.var(omega_tilde_1_reel_LASSO_ss[1000:])), color = 'purple')
axs[0].set_ylabel('w_tilde_1')
axs[0].legend()

axs[1].plot(range(9001), omega_tilde_2ss_d2_reel[1000:], label="ZV 2 {:.2f}".format(np.var(omega_tilde_2ss_d2_reel[1000:])), color = 'grey')
axs[1].plot(range(9001), omega_tilde_2_reel_LASSO_ss[1000:], label="LASSO {:.3f}".format(np.var(omega_tilde_2_reel_LASSO_ss[1000:])), color = 'purple')
axs[1].set_ylabel('w_tilde_2')
axs[1].legend()

axs[2].plot(range(9001), omega_tilde_3ss_d2_reel[1000:], label="ZV 2 {:.4f}".format(np.var(omega_tilde_3ss_d2_reel[1000:])), color = 'grey')
axs[2].plot(range(9001), omega_tilde_3_reel_LASSO_ss[1000:], label="LASSO {:.5f}".format(np.var(omega_tilde_3_reel_LASSO_ss[1000:])), color = 'purple')
axs[2].set_ylabel('w_tilde_3')
axs[2].legend()

plt.suptitle("Données réelles : \n Les trajectoires des omega_tildes avec une régression non pénalisée et une régression pénalisée")